Preamble here:

In [3]:
#NeuroDiffEq can be used without running this cell. This is for loading Julia and installing/importing PySR for an example at the end.
try:
    import pysr
    pysr.julia_helpers.init_julia()

except ImportError:
    import subprocess
    import sys

    print("Installing Julia and PySR. This might take some time...")

    shell_script = '''
    # @title Install Julia and PySR (Takes some time)
    %%shell
    set -e

    #---------------------------------------------------#
    JULIA_VERSION="1.8.2" # any version ≥ 0.7.0
    JULIA_PACKAGES="IJulia BenchmarkTools"
    JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
    JULIA_NUM_THREADS=2
    #---------------------------------------------------#

    if [ -z `which julia` ]; then
        # Install Julia
        JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
        echo "Installing Julia $JULIA_VERSION..."
        BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
        URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
        wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
        tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
        rm /tmp/julia.tar.gz

        # Install Packages
        nvidia-smi -L &> /dev/null && export GPU=1 || export GPU=0
        if [ $GPU -eq 1 ]; then
        JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
        fi
        for PKG in `echo $JULIA_PACKAGES`; do
        echo "Installing Julia package $PKG..."
        julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
        done

        # Install kernel and rename it to "julia"
        echo "Installing IJulia kernel..."
        julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
            "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
        KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
        KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
        mv -f $KERNEL_NAME "$KERNEL_DIR"/julia

        echo ''
        echo "Successfully installed `julia -v`!"
    fi
    '''

    process = subprocess.Popen(shell_script, shell=True, executable='/bin/bash')
    process.wait()

    subprocess.run([sys.executable, '-m', 'pip', 'install', '-U', 'pysr'])
    subprocess.run([sys.executable, '-m', 'pysr', 'install'])


[juliapkg] Found dependencies: c:\users\nachi\onedrive\desktop\cphysics\c_physics_projects\.pysrenv\lib\site-packages\juliacall\juliapkg.json
[juliapkg] Found dependencies: c:\users\nachi\onedrive\desktop\cphysics\c_physics_projects\.pysrenv\lib\site-packages\juliapkg\juliapkg.json
[juliapkg] Found dependencies: c:\users\nachi\onedrive\desktop\cphysics\c_physics_projects\.pysrenv\lib\site-packages\pysr\juliapkg.json
[juliapkg] Locating Julia =1.10.0, ^1.10.3
[juliapkg] Using Julia 1.10.3 at C:\Users\nachi\AppData\Local\Programs\julia-1.10.3\bin\julia.EXE
[juliapkg] Using Julia project at c:\Users\nachi\OneDrive\Desktop\cphysics\c_physics_projects\.pysrenv\julia_env
[juliapkg] Installing packages:
           julia> import Pkg
           julia> Pkg.Registry.update()
           julia> Pkg.add([Pkg.PackageSpec(name="PythonCall", uuid="6099a3de-0909-46bc-b1f4-468b9a2dfc0d"), Pkg.PackageSpec(name="SymbolicRegression", uuid="8254be44-1295-4e6a-a16d-46603ac705cb"), Pkg.PackageSpec(name="Serial

c:\Users\nachi\OneDrive\Desktop\cphysics\c_physics_projects\.pysrenv\Lib\site-packages\pysr\deprecated.py:19: FutureWarning: The `init_julia` function has been removed. Julia is now initialized automatically at import time.
  warnings.warn(


In [ ]:
try:
    import neurodiffeq
except:
    import os
    os.system("pip install neurodiffeq")

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from neurodiffeq.neurodiffeq import safe_diff as diff
from neurodiffeq.ode import solve, solve_system
from neurodiffeq.solvers import Solver1D 
from neurodiffeq.monitors import Monitor1D #Helps visualize the loss and fit at certain epochs
from neurodiffeq.conditions import IVP

import matplotlib.pyplot as plt
from neurodiffeq.callbacks import ActionCallback
from neurodiffeq.networks import FCNN # fully-connect neural network
import torch.nn as nn                 # PyTorch neural network module
from neurodiffeq.generators import Generator1D
from neurodiffeq.conditions import NoCondition, IVP
import copy

%matplotlib inline

Start with an explanation of PINNs: 

    1) What are regular neural networks: stuff about training functions, tuning weights, finding minima, minimizing loss with different loss functions
    2) What makes PINNs different?
    3) What is NeuroDiffEq? What makes it special? Difference from autograd.



Can start with the table from the start of the video explaining the use cases of neurodiffeq. 

Move onto basic examples of neurodiffeq:

Start with simple harmonic oscillator
Talk about boundary conditions and briefly go over all of the different boundary that NeuroDiffEq supports. eg. Neumann, Dirichlet, etc. 

Talk about the generator - what is it, its uses (this is part of the final demo combining PySR and NeuroDiffEq so it'll be useful. Very short thing.)


## The Diff Function

This is at the core of NeuroDiffEq. We use this to write the differential equations that we pass to the solver. 

In [ ]:
t = torch.linspace(0,2*torch.pi,50,requires_grad=True)
t = torch.reshape(t,(-1,1)) #The -1 reduces the dimension to a single dimension. This turns the input into a single column vector
u = torch.sin(t)
du_dt = neurodiffeq.diff(u,t)

plt.figure(figsize=(10,5))
plt.plot(t.detach().numpy(),u.detach().numpy(),label="Original function")
plt.plot(t.detach().numpy(),du_dt.detach().numpy(),label= "Differentiated function")
plt.legend()

## 1. Train for the Potential function

The problem consists on solving the differential equation using `neurodiffeq` and recover the potential $V$. We denote $x$ as the time and $y$ as the displacement to make it easier to work with the variables. $m$ is the mass of the oscillator and $k$ is the spring constant.

$$w = \sqrt \frac{k}{m}$$

$$m\ddot y + ky = 0 $$

To simplify the process we will take $m=1$

$$ \frac{d^2y}{dx^2} + \frac{dV}{dy} = 0 $$



When the potential, $V$, is unknown. The case at hand is the harmonic oscilator for which $dV/dy = ky$, where $w=2$, $k=4$, $\;\boxed{\Rightarrow V(y) = (k/2)y^2 = 2y^2}$. For this diff eq and this potential, the solution is $\boxed{y(x) = \sin(wx)}$

To deal with ODE systems, we **split it into a system of ODEs**:

$$ \frac{d(vy)}{dx} + \frac{dV}{dy} = 0 $$

$$ vy - \frac{dy}{dx} = 0 $$

**We define NNs for both systems** that take $x$ and output $y(x)$ (`net_xy`) and $x$ to output $dy/dx$ (`net_xvy`) respectively. They also define an auxiliary NN that takes $y$ and outputs $V(y)$ (`net_yV`). `net_xy` and `net_xvy` are the NNs of the solver, but `net_xvy` is internal to the system and has to be accessed via callbacks. **The interesting thing about this is that the NN is being trained on data from x and y, and manages to optimize well for V(y) as well**.

**Callback:** a feature to inject custom code into the training loop of a NN to perform various actions at different stages of the training process without modifying the core training loop.

First we define the data we want to train on 

In [ ]:
## Generate data
w = 2.0
k = 4.0 

x_true = np.linspace(0, np.pi, 256)
# True data is y = sin(wx) + noise
y_true =  np.sin(w*x_true) + 0.2*np.random.randn(256)

train_gen = Generator1D(size=128,  t_min=0.0, t_max=np.pi, method='equally-spaced-noisy')
valid_gen = Generator1D(size=128, t_min=0.0, t_max=np.pi, method='equally-spaced')

The generator objects are called during training to generate data at the sampled points. We can see this using their .get_examples() method.This makes the process of generating data for ODEs much easier. There are various different types of generators in NeuroDiffEq like 1D, 2D, 3D etc.

In [ ]:
train_gen.get_examples()

Next we define our network and system of equations

In [ ]:
## x --> y --> V
# Specify the network architecture
# Each Fully Connected Neural Netwrok (FCNN) has 1 input layer, 2 hidden layers (each with 64 units), and 1 output layer
net_xy = FCNN(
    hidden_units=(64, 64,), actv=nn.Tanh
) # this one takes x and outputs y(x)

net_xvy = FCNN(
    hidden_units=(64, 64,), actv=nn.Tanh
) # this takes x and outputs vy=dy/dx

net_yV = FCNN(
    hidden_units=(64, 64,), actv=nn.Tanh
) # This one takes y and outputs V(y)


nets = [net_xy, net_xvy] # this will be the diff eq in NN terms

def system(y, vy, x):
  # We want to solve y'' + dV/dy = 0 but we frame it as a system of first order equations
  V = net_yV(y)
  eq1 = diff(V,y) + diff(vy,x,order = 1)
  eq2 = diff(y,x) - vy
  return [eq1, eq2]


conditions = [
    IVP(t_0=0.0, u_0=0.0),
    IVP(t_0=0.0, u_0=w),
]

Now we define the Solver. This is similar to Solvers from sklearn and keras. We instantiate the solver by specifying the ODE/PDE system, initial/boundary conditions, problem domain, etc.

In [ ]:
#Define Solver

class Solver_withMSE(Solver1D):
  def __init__(self, x_true = None, y_true = None, alpha_init = 1, **kwargs):
    super().__init__(**kwargs) # inherit Solver1D __init__
    self.x_true = torch.tensor(x_true, requires_grad = True).view(-1,1) #Same as the reshape shown earlier
    self.y_true = torch.tensor(y_true).view(-1,1)
    self.alpha_init = alpha_init
    self.b = -4

  # overwrite the `additional_loss` method from Solver1D to consider the mse as a term in the loss function
  def additional_loss(self, residual, funcs, coords):
    epochs = self.global_epoch
    max_epochs = self._max_local_epoch
    frac = epochs/max_epochs
    # alpha does annealing of the coefficient of mse loss becase we want the coefficient of the MSE loss to be higher when we start training
    # annealing: Allowing for the trainability to vary with the epoch No. From high variability to low.
    alpha = (self.alpha_init-1)*np.exp(self.b*frac) - (self.alpha_init-1)*np.exp(self.b)*frac + 1
    batch = [self.x_true]

    funcs = [
    self.compute_func_val(n, c, *batch) for n, c in zip(self.nets, self.conditions)
    ] # Computes the function value (the neural net outputs in this case) evaluated 
      # on the points specified by coordinates (the batch/ x_true in this case)

    loss_mse = ((funcs[0]-self.y_true)**2).mean() # simple mean squared error loss. `funcs[0]` is the y prediction. 
                                                  #`funcs[1]` would be the dy/dx prediction.
    return alpha*loss_mse
  
  # We start by emphasizing MSE at first with the high alpha to allow the network to quickly learn the true solution.
  # Alpha is lowered overtime so that MSE is deemphasized, allowing the network to focus more on satisfying the ODE.


#Allows us to use multiple optimizers
class DualOptimizer():
  def __init__(self, optimizers):
    self.optimizers = optimizers

  def step(self, closure = None):
    for opt in self.optimizers:
      opt.step(closure)

  def zero_grad(self):
    for opt in self.optimizers:
      opt.zero_grad()


opt_yx = torch.optim.Adam(set([p for net in nets for p in net.parameters()]), lr=1e-3, weight_decay=0.0001)
opt_potential = torch.optim.Adam(net_yV.parameters(), lr=1e-3)

dualoptimizer = DualOptimizer([opt_yx, opt_potential])

class DoSchedulerStep(ActionCallback):
    def __init__(self, scheduler):
        super().__init__()
        self.scheduler = scheduler

    def __call__(self, solver):
        self.scheduler.step()

class BestValidationCallback(ActionCallback):
    def __init__(self):
        super().__init__()
        self.best_potential = None

    def __call__(self, solver):
        if solver.metrics_history['train_loss'][-1] <= solver.lowest_loss:
            self.best_potential = copy.deepcopy(net_yV)  #Saves the best model of the potential


#scheduler = torch.optim.lr_scheduler.StepLR(opt, step_size=1200, gamma=0.95) #This changes the learning rate during the training process
#scheduler_cb = DoSchedulerStep(scheduler=scheduler)
potential_cb = BestValidationCallback()


solver = Solver_withMSE(
    x_true = x_true,
    y_true = y_true,
    alpha_init = 10,
    ode_system=system,
    conditions=conditions,
    t_min=0,
    t_max=np.pi,
    optimizer = dualoptimizer,
    train_generator=train_gen,
    valid_generator=valid_gen,
    nets=nets,
    n_batches_valid=0,
)


In [ ]:
#Fitting the solver

solver.fit(max_epochs=15000, callbacks=[potential_cb])#, scheduler_cb])


In [ ]:
history = solver.metrics_history

plt.plot(history['train_loss'], label = 'train_loss')
#plt.plot(history['valid_loss'], label = 'valid_loss')
plt.legend()
plt.yscale('log')

In [ ]:
solution = solver.get_solution(best=True)
xs = np.linspace(0, np.pi, 256)
ys = solution(xs)[0]
mse1 = ((ys.cpu().detach()-y_true)**2).mean()

#Since ys is a tensor as it is the output of the NN, we need .cpu().detach().numpy() to turn it into an object we can pass to the ploot func

plt.plot(xs, ys.cpu().detach().numpy(), label = 'y_pred {:.4f}'.format(mse1), color = 'orange')
plt.scatter(x_true, y_true, label = 'y_data', lw = 0.1, alpha = 0.6)
plt.legend()
plt.title('y vs x')
plt.show()

In [ ]:
solution = solver.get_solution(best=True)
xs = torch.tensor(np.linspace(0, np.pi, 100)).view(-1,1)
ys = solution(xs)[0].view(-1,1)
Vs = potential_cb.best_potential(ys)
dVdy = diff(Vs, ys)
plt.plot(ys.cpu().detach().numpy(), Vs.cpu().detach().numpy() - Vs[0].item(), label = '$V(y)$')
#Exact potential = 1/2*k*y^2
plt.plot(ys.cpu().detach().numpy(), 0.5*(k)*ys.pow(2).cpu().detach().numpy(), label = '$Exact V$')
plt.legend()
plt.title('V vs y')
plt.show()

In [ ]:
solution = solver.get_solution(best=True)
xs = torch.tensor(np.linspace(0, np.pi, 100)).view(-1,1)
ys = solution(xs)[0].view(-1,1)
Vs = potential_cb.best_potential(ys)


X = ys.cpu().detach().numpy()
y = Vs.cpu().detach().numpy() - Vs[0].item()

from pysr import PySRRegressor

model_SR = PySRRegressor(
    niterations=50,  # < Increase me for better results
    binary_operators=["+", "*"],
    unary_operators=[
        "cos",
        "exp",
        "sin",
        "inv(x) = 1/x",
        # ^ Custom operator (julia syntax)
    ],
    extra_sympy_mappings={"inv": lambda x: 1 / x},
    # ^ Define operator for SymPy as well
    loss="loss(prediction, target) = (prediction - target)^2",
    # ^ Custom loss function (julia syntax)
)
model_SR.set_params(extra_torch_mappings = {'inv(x)': lambda x: 1/x})

In [ ]:
model_SR.fit(X, y)

## 2. Train PINN on candidate potentials without data then validate with data

In [ ]:
top7_candidates = model_SR.pytorch(index = list(range(7))) # ?? this retrieves a torch-like symbolic expression that can be used to evaluate tensors
min_loss = 1e5 # this is just initializing the min_loss to a large value. You could do np.inf
min_loss_cand = None
for cand in top7_candidates:
  net_xy_cand = FCNN(
    hidden_units=(64, 64,), actv=nn.Tanh
  )

  def system_cand(y, x):
    V = cand(y).view(-1,1) # calculating the potential given by symbolic expression (that is trainable??)
    if V.shape == torch.Size([1, 1]):
      V = V * torch.ones_like(y)
    eq1 = diff(V,y) + diff(y,x,order = 2) # this is the whole DE of 2nd order
    return [eq1]

  nets_cand = [net_xy_cand]
  opt_yx_cand = torch.optim.Adam(set([p for net in nets_cand for p in net.parameters()]), lr=1e-3)
  conditions_cand = [
    IVP(t_0=0.0, u_0=0.0, u_0_prime = w),
  ]

  solver_cand = Solver1D(
    ode_system=system_cand,
    conditions=conditions_cand,
    t_min=0,
    t_max=np.pi,
    optimizer = opt_yx_cand,
    train_generator=train_gen,
    valid_generator=valid_gen,
    nets=nets_cand,
    n_batches_valid=0,
  )

  solver_cand.fit(max_epochs = 2000)
  solution_cand = solver_cand.get_solution(best = True)
  y_pred = solution_cand(x_true).cpu().detach().numpy()
  mse = ((y_pred - y_true)**2).mean()
  if mse < min_loss:
    min_loss = mse
    min_loss_cand = cand
  print('Candidate potential :',cand._expression_string, 'MSE with data : {:.7f}'.format(mse))
  plt.plot(x_true, y_pred, label = 'pred')
  plt.scatter(x_true, y_true, label = 'data', color = 'red', alpha = 0.3, lw = 0.1)
  plt.legend()
  plt.show()
  print('================================================')

print('LOWEST MSE OBTAINED BY: ',cand._expression_string, ' = ', min_loss)
print('================================================')

## 3. Finetuning candidates on the trained net

In [ ]:
print('parameters before training: ',list(cand.named_parameters()))
print('expression before training: ',cand._expression_string)

In [ ]:
top7_candidates = model_SR.pytorch(index = list(range(7)))

for cand in top7_candidates:
  net_xy_cand = FCNN(
    hidden_units=(64, 64,), actv=nn.Tanh
  )

  def system_cand(y, x):
    V = cand(y).view(-1,1)
    if V.shape == torch.Size([1, 1]):
      V = V * torch.ones_like(y)
    eq1 = diff(V,y) + diff(y,x,order = 2)
    return [eq1]

  print('parameters before training: ',list(cand.named_parameters()))
  print('expression before training: ',cand._expression_string)
  nets_cand = [copy.deepcopy(net_xy)] # changed
  opt_yx_cand = torch.optim.Adam(set([p for net in nets_cand for p in net.parameters()]), lr=1e-3, weight_decay = 0.0001)
  conditions_cand = [
    IVP(t_0=0.0, u_0=0.0, u_0_prime = 2.0),
  ]
  if len(list(cand.parameters()))>0:
    opt_cand = torch.optim.Adam(cand.parameters(), lr=1e-2)
    dualoptimizer_cand = DualOptimizer([opt_yx_cand, opt_cand])
    opt_final = dualoptimizer_cand
  else:
    opt_final = opt_yx_cand

  solver_cand = Solver_withMSE(
    x_true = x_true,
    y_true = y_true,
    ode_system=system_cand,
    conditions=conditions_cand,
    t_min=0,
    t_max=np.pi,
    optimizer = opt_final,
    train_generator=train_gen,
    valid_generator=valid_gen,
    nets=nets_cand,
    n_batches_valid=0,
  )
  solver_cand.fit(max_epochs = 2000)
  print('parameters after training: ',list(cand.named_parameters()))
  solution_cand = solver_cand.get_solution(best = True)
  y_pred = solution_cand(x_true).cpu().detach().numpy()
  print('Candidate potential :',cand._expression_string, 'MSE : {:.7f}'.format(((y_pred - y_true)**2).mean()), 'Lowest Loss : {:.7f}'.format(solver_cand.lowest_loss))#, solver_cand.lowest_loss)
  plt.plot(x_true, y_pred, label = 'pred')
  plt.scatter(x_true, y_true, label = 'data', color = 'red', alpha = 0.3, lw = 0.1)
  plt.plot(x_true, np.sin(2*x_true), label = 'exact data')
  plt.legend()
  plt.show()
  print('================================================')